In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.optimizers import Adam,SGD, RMSprop 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras import regularizers


In [18]:
img_size = 48
train_dir = "Data/train"
test_dir = "Data/test"

# Generate random transformation from Data to increase Dataset and prevent overfitting
train_datagen = ImageDataGenerator( width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         horizontal_flip = True,
                                         rescale = 1./255,
                                         zoom_range = 0.3,
                                         validation_split = 0.2)

val_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(directory= train_dir,
                                                    target_size = (img_size, img_size),
                                                    shuffle = True,
                                                    batch_size = 64,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical"
                                                    )
validation_generator = val_datagen.flow_from_directory (directory= test_dir,
                                                        target_size = (img_size, img_size),
                                                        shuffle = True,
                                                        batch_size = 64,
                                                        color_mode = "grayscale",
                                                        class_mode = "categorical")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [35]:
batch_size = 64

In [26]:
# Building CNN Architecture
    input_size = (48,48,1)
    model = Sequential()   

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape =input_size))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(classes, activation='softmax'))

    model.compile(
        optimizer = Adam(lr=0.0001, decay=1e-6), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
          )

In [22]:
classes = 7
get_model((48,48,1), classes)

In [28]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 48, 48, 64)        18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 22, 22, 256)      

In [33]:
# Compose model callback metrics
import datetime
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
chk_path = 'ferNet.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [ ]:
#Train the model
steps_per_epoch = train_generator.n// batch_size
validation_steps = validation_generator.n//batch_size

history = model.fit(x=train_generator,
                 validation_data=validation_generator,
                 epochs=60,
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

Epoch 1/60
448/448 [==============================] - ETA: 0s - loss: 4.3438 - accuracy: 0.2429
Epoch 00001: val_loss improved from inf to 7.55287, saving model to ferNet.h5
448/448 [==============================] - 1766s 4s/step - loss: 4.3438 - accuracy: 0.2429 - val_loss: 7.5529 - val_accuracy: 0.1797
Epoch 2/60
448/448 [==============================] - ETA: 0s - loss: 3.5652 - accuracy: 0.2859 
Epoch 00002: val_loss improved from 7.55287 to 3.18680, saving model to ferNet.h5
448/448 [==============================] - 5127s 11s/step - loss: 3.5652 - accuracy: 0.2859 - val_loss: 3.1868 - val_accuracy: 0.3597
Epoch 3/60
448/448 [==============================] - ETA: 0s - loss: 3.0684 - accuracy: 0.3118
Epoch 00003: val_loss improved from 3.18680 to 2.73105, saving model to ferNet.h5
448/448 [==============================] - 1610s 4s/step - loss: 3.0684 - accuracy: 0.3118 - val_loss: 2.7311 - val_accuracy: 0.3821
Epoch 4/60
448/448 [==============================] - ETA: 0s - loss:

In [ ]:
# Plot Model accuracy and loss
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#Save model to file
model.save_weights('model .h5')